In [29]:

# Step 1 - Complete your initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.
from bs4 import BeautifulSoup
import pymongo
from splinter import Browser
import pandas as pd
import time

# NASA Mars News

In [30]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text.
#Assign the text to variables that you can reference later.
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    import os
    if os.name=="nt":
        executable_path = {'executable_path': './chromedriver.exe'}
    else:
        executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [31]:
def nasa_mars_news():
    browser = init_browser()
    nasa_url = 'https://mars.nasa.gov/news'
    browser.visit(nasa_url)
    time.sleep(5)
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'lxml')
    nasa_news  = []
    for slides in soup.find_all('li', class_='slide'):
        news_title = slides.find_all('div', class_='content_title')[0].text
        news_p = slides.find_all('div', class_='article_teaser_body')[0].text
        nasa_news =  [news_title, news_p]
        break
    browser.quit()
    return nasa_news 

In [32]:
def featured_mars_img():
    browser = init_browser()
    jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(jpl_url)
    time.sleep(5)
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'lxml')
    image_url = soup.find('article', class_='carousel_item')['style']
    mars_image_url =image_url.replace('background-image: url(', '').replace(');','')[1:-1]
    main_url = 'https://www.jpl.nasa.gov'
    featured_image_url = main_url + mars_image_url
    browser.quit()
    return featured_image_url  


In [33]:
def mars_weather_twitter():
    browser = init_browser()
    twitter_url = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(twitter_url)
    time.sleep(5)
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    mars_weather = soup.find_all('span')
    for tweet in mars_weather:
        if 'InSight sol' in tweet.text:
            mars_weather = tweet.text
            break
    browser.quit()
    return mars_weather 


In [34]:
def mars_facts_table():
    facts_url = 'https://space-facts.com/mars/'
    facts_tables = pd.read_html(facts_url)
    mars_facts = facts_tables[0]
    mars_facts.columns = ["Description", "Value"]
    mars_facts = mars_facts.set_index("Description")
    mars_table = mars_facts.to_html()
    return mars_table

In [35]:
def mars_hems_imgs():
    browser = init_browser()
    hems_url ='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(hems_url)
    time.sleep(5)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all('div', class_='item')
    hem_title_urls  = []
    main_url = 'https://astrogeology.usgs.gov'  
    for result in results:
        title = result.find('h3').text
        image_url = result.find('a', class_='itemLink product-item')['href']
        image_url = main_url+image_url
        browser.visit(image_url)
        time.sleep(5)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        result = browser.links.find_by_text('Sample').first
        enhanced_url = result['href']
        hem_title_urls.append({"Title" : title, "Image_URL" : enhanced_url}) 
    browser.quit()  
    return hem_title_urls


In [36]:
def scrape_info():
    
    mars_info = {}
    nasa_news = nasa_mars_news()
    mars_info["title"] = nasa_news[0]
    mars_info["par"] = nasa_news[1]
    mars_info["featured_image_url"] = featured_mars_img()
    mars_info["mars_weather"] = mars_weather_twitter()
    mars_info["mars_table"] = mars_facts_table()
    mars_info["hem_title_urls"] = mars_hems_imgs()

    # Return results
    return mars_info

In [4]:
# browser = Browser('chrome', **executable_path, headless=True)

In [5]:
# URL of page to be scraped
# url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
# browser.visit(url)

In [37]:

# # HTML object
# html = browser.html
# # Parse HTML with Beautiful Soup
# soup = BeautifulSoup(html, 'html.parser')

# # Examine the results, then determine element that contains sought info
# info = soup.find_all('div', class_="list_text")

# #Create a container for the data that will be scraped
# news_title = []
# news_p = []

# # Loop through returned results
# for x in info:
#     title = x.find('a').text.strip()
#     paragraph = x.find( class_='article_teaser_body').text.strip()
    
#     news_title.append(title)
#     news_p.append(paragraph)
        
#     print('----------')
#     print(title)
#     print(paragraph)
        
# print('-----------')
# print(f"News title count: {len(news_title)}, and new paragraph count: {len(news_p)} on this page")

In [38]:
# #Save to CSV file
# news_mars = pd.DataFrame({"News Title": news_title,"News Paragraph" : news_p
# })

# news_mars.head()

In [39]:
# news_mars.to_csv("Saved Output/news_mars.csv")

# JPL Mars Space Images - Featured Image

In [8]:
# # URL of page to be scraped
# url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars#submit'
# browser.visit(url)

In [9]:

# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Examine the results, then determine element that contains sought info
info = soup.find_all('div', class_="img")


#Assign the url string to a variable called featured_image_url
#Create a container for the data that will be scraped
featured_image_url= []

# Loop through returned results
for x in info:

    image = x.find('img')
    image_loc = image['src']
    
    print('https://www.jpl.nasa.gov' + image_loc)  
    
    featured_image_url.append('https://www.jpl.nasa.gov' + image_loc)
    print('--------')
        
    
print('--------')
print(f"Count of images in the webpage {len(featured_image_url)}")

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23960-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23959-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23958-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23957-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23956-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23950-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23949-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23948-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23900-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23947-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23899-640x350.jpg
--------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23946-640x350.jpg
-

In [10]:
#Inspect data
print (featured_image_url)

['https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23960-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23959-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23958-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23957-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23956-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23950-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23949-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23948-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23900-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23947-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23899-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23946-640x350.jpg', 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23897

In [11]:
#import dataframe to convert the list to df
from pandas import DataFrame

In [12]:
featured_df = DataFrame (featured_image_url, columns=['Featured Mars Images'])
print(featured_df)

                                 Featured Mars Images
0   https://www.jpl.nasa.gov/spaceimages/images/wa...
1   https://www.jpl.nasa.gov/spaceimages/images/wa...
2   https://www.jpl.nasa.gov/spaceimages/images/wa...
3   https://www.jpl.nasa.gov/spaceimages/images/wa...
4   https://www.jpl.nasa.gov/spaceimages/images/wa...
5   https://www.jpl.nasa.gov/spaceimages/images/wa...
6   https://www.jpl.nasa.gov/spaceimages/images/wa...
7   https://www.jpl.nasa.gov/spaceimages/images/wa...
8   https://www.jpl.nasa.gov/spaceimages/images/wa...
9   https://www.jpl.nasa.gov/spaceimages/images/wa...
10  https://www.jpl.nasa.gov/spaceimages/images/wa...
11  https://www.jpl.nasa.gov/spaceimages/images/wa...
12  https://www.jpl.nasa.gov/spaceimages/images/wa...
13  https://www.jpl.nasa.gov/spaceimages/images/wa...
14  https://www.jpl.nasa.gov/spaceimages/images/wa...
15  https://www.jpl.nasa.gov/spaceimages/images/wa...
16  https://www.jpl.nasa.gov/spaceimages/images/wa...
17  https://www.jpl.nasa.gov

In [13]:
#Save the df as CSV file 
featured_df.to_csv("Saved Output/featured_image_url.csv", index=False)

# Mars Weather

In [14]:
# URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
time.sleep(2)

In [15]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Examine the results, then determine element that contains sought info
info = soup.find_all('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0")

#Create a container for the data that will be scraped
mars_weather = []
   
for x in info: 
    weather = x.text.strip()
    if (weather!="" and weather[0]=='I'):
        mars_weather.append(weather)
        print(weather)
        print('---------')
        
     
    print(f"The count of tweet text for Mars's weather is {len(mars_weather)}")

The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weather is 0
The count of tweet text for Mars's weath

In [16]:
#Save the tweet text for the weather report as a variable called mars_weather
mars_weather_df = DataFrame (mars_weather, columns=['Mars Weather Tweets'])
print(mars_weather_df)

                                 Mars Weather Tweets
0  InSight sol 566 (2020-06-29) low -88.4ºC (-127...
1  It's our birthday! On this day in 1976, a sign...
2  InSight sol 565 (2020-06-28) low -89.6ºC (-129...
3  InSight sol 564 (2020-06-28) low -89.3ºC (-128...


In [17]:
#Save the df as CSV file 
mars_weather_df.to_csv("Saved Output/mars_weather.csv", index=False)


# Mars Facts

In [18]:
# URL of page to be scraped
url = 'https://space-facts.com/mars/'

In [19]:
#Read and inspect the data
mars_facts = pd.read_html(url)
mars_facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [20]:
#Convert list to dataframe
mars_facts_df = mars_facts[0]
mars_facts_df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [21]:
#Rename the column titles
mars_facts_df = mars_facts_df.rename(columns={0:"Unit", 1:"Measure"})
mars_facts_df

,Unit,Measure
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [22]:
#Save the dataframe to html
mars_facts_html = mars_facts_df.to_html('Saved Output/mars_facts.html')

# Mars Hemispheres

In [23]:
# URL of page to be scraped
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [24]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [25]:
# Examine the results, then determine element that contains sought info
info = soup.find_all('div', class_="item")
info

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>,
 <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiapa

In [26]:
#Create a container for the data that will be scraped
hemisphere_details = []
   
#Loop through the returned results for Hemisphere name    
for x in info:

    description = x.find('div', class_="description")
    h3 = description.find('h3').text.strip()
    
    a = x.find('a')
    href = a['href']
    
    hemisphere_name_url = 'https://astrogeology.usgs.gov' + href
    print(hemisphere_name_url)
    
    hemisphere_details.append('https://astrogeology.usgs.gov' + href)
    print('--------')   


https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
--------
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
--------
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
--------
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced
--------


In [27]:
hemisphere_details

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [28]:
import requests
import time


hemisph_images = {}
   
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve all elements that contain articles information
info = soup.find_all('div', class_="item")

for x in info:

    description = x.find('div', class_="description")
    h3 = description.find('h3').text.strip()
    
    link = x.find('a')
    href = link['href']
    
    links = 'https://astrogeology.usgs.gov' + href
    print(links)
    
    browser.visit(links)
    time.sleep(2)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    info_image = soup.find_all('img', class_="wide-image")
    
    src =  info_image[0]['src']
    
    hemisph_images.update( {h3 : 'https://astrogeology.usgs.gov' + src} )
    print('title:',h3)
    print('image_url:','https://astrogeology.usgs.gov' + src)
    print('-----------')
        
print(f"The {len(hemisph_images)} images and their links")

https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
title: Cerberus Hemisphere Enhanced
image_url: https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
-----------
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
title: Schiaparelli Hemisphere Enhanced
image_url: https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
-----------
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
title: Syrtis Major Hemisphere Enhanced
image_url: https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
-----------
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced
title: Valles Marineris Hemisphere Enhanced
image_url: https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg
-----------
The 

In [28]:
mars_info = {}
nasa_news = news_mars()
mars_info["title"] = news_title
mars_info["par"] = news_p
mars_info["featured_image_url"] = featured_df()
mars_info["mars_weather"] = mars_weather_df()
mars_info["mars_table"] = mars_facts_df()
mars_info["hem_title_urls"] = hemisph_images()



TypeError: 'DataFrame' object is not callable